# __ETL__ _(Extract, Transform, Load)_

## Introducción

Este notebook se enfoca en el proceso de **ETL** utilizando datos extraídos de las plataformas Yelp y Google Maps. Este proceso implica una _extracccion,transformación y carga_ de los datos con el objetivo de prepararlos para análisis posteriores. Este paso es crucial en cualquier proyecto de ciencia de datos para garantizar la calidad y utilidad de los datos.

## Configuraciones Globales e Importaciones

En esta sección, se instalan e importan todas las librerías y/o módulos necesarios para el proceso ETL (Extract, Transform, Load) y se establecen configuraciones globales de ser requerido. Se utilizan las siguientes librerías y herramientas:

In [ ]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Utilizamos google drive para poder trabajar en linea con el equipo del proyecto

In [ ]:
import warnings
warnings.filterwarnings("ignore") # Se utiliza para gestionar las advertencias y mantener el código limpio.

In [ ]:
import os # Proporciona funciones para interactuar con el sistema operativo.
import pandas as pd # Una librería de análisis de datos.
import json # Se utiliza para trabajar con datos en formato JSON.

# YELP

**Dataset:** CHECKIN.JSON

DECLARACIÓN DE LA RUTA DE LOS DATA SET

In [ ]:
# Ruta local completa del archivo checkin.json
ruta_checkin = '/content/drive/MyDrive/Colab-Notebooks/Yelp/checkin.json'

In [ ]:
checkin_yelp = []

with open(ruta_checkin, "r", encoding='utf-8') as filejson:
    # Procesa cada línea del archivo como un objeto JSON
    for linea in filejson:
        # Intenta cargar la línea como un objeto JSON
        objeto = json.loads(linea)
        checkin_yelp.append(objeto)

# Convierte la lista de diccionarios a un DataFrame de Pandas
checkin_yelp = pd.DataFrame(checkin_yelp)

In [ ]:
#MUestra las primero 5 filas del dataframe
checkin_yelp.head(5)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [ ]:
#Muesta la informacion del dataframe
checkin_yelp.info()
checkin_yelp.shape[0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


131930

Notamos que la columna *DATE* no tiene un formato correcto que se adapte a nuestra base de dato por eso procederemos a realizar las siguientes modificaciones

In [ ]:
# Divide la columna 'date' en una lista de fechas
checkin_yelp['date'] = checkin_yelp['date'].str.split(', ')

In [ ]:
# Convierte la lista de fechas en filas separadas
checkin_yelp = checkin_yelp.explode('date').reset_index(drop=True)

In [ ]:
# Convierte la columna 'date' al formato datetime
checkin_yelp['date'] = pd.to_datetime(checkin_yelp['date'])

In [ ]:
# Ordena el DataFrame por 'business_id' y 'date'
checkin_yelp = checkin_yelp.sort_values(by=['business_id', 'date'])

In [ ]:
# Convertir la columna 'date' a tipo datetime
checkin_yelp['date'] = pd.to_datetime(checkin_yelp['date'])

# Extraer solo la parte de la fecha (año-mes-día)
checkin_yelp['date'] = checkin_yelp['date'].dt.date

# Mostrar las primeras filas del DataFrame con la fecha formateada
checkin_yelp.head(10)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09
5,---kPU91CF4Lq2-WlRu9Lw,2021-01-20
6,---kPU91CF4Lq2-WlRu9Lw,2021-04-30
7,---kPU91CF4Lq2-WlRu9Lw,2021-05-25
8,---kPU91CF4Lq2-WlRu9Lw,2021-08-06
9,---kPU91CF4Lq2-WlRu9Lw,2021-10-02


In [ ]:
#MUestra las primero 5 filas del dataframe
checkin_yelp.head(5)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,2020-03-13
1,---kPU91CF4Lq2-WlRu9Lw,2020-06-02
2,---kPU91CF4Lq2-WlRu9Lw,2020-07-24
3,---kPU91CF4Lq2-WlRu9Lw,2020-10-24
4,---kPU91CF4Lq2-WlRu9Lw,2020-12-09


### **VERIFICAMOS CANTIDAD DE FILAS**

In [ ]:
checkin_yelp.count()

business_id    13356875
date           13356875
dtype: int64

### **VERIFICAMOS NULOS**

In [ ]:
def contar_nulos(dataframe):

    # Obtener la cantidad de valores nulos por columna
  nulos_por_columna = dataframe.isnull().sum()

  print("Cantidad de valores nulos por columna:\n", nulos_por_columna)

In [ ]:
#Cuantos nulos hay en cada columna
contar_nulos(checkin_yelp)

Cantidad de valores nulos por columna:
 business_id    0
date           0
dtype: int64


### **VERIFICAMOS DUPLICADOS**

In [ ]:
# Revisa si hay filas duplicadas en el DataFrame
duplicados = checkin_yelp.duplicated()

# Muestra las filas duplicadas
print(checkin_yelp[duplicados])


                     business_id        date
17        --0iUa4sNDFiZFrAdIWhZQ  2013-08-13
27        --7PUidqRWpRSpXebiyxTg  2012-04-24
63        --8IbOsAAxjKRoYsBFL-PA  2015-08-07
70        --8IbOsAAxjKRoYsBFL-PA  2015-12-23
78        --8IbOsAAxjKRoYsBFL-PA  2016-09-23
...                          ...         ...
13356642  zzbZtgPYZS8sTIWQH6DwEw  2017-02-25
13356733  zziDpuuJw-Km1J4BaGpBKA  2015-05-10
13356741  zziDpuuJw-Km1J4BaGpBKA  2016-12-24
13356810  zznJox6-nmXlGYNWgTDwQQ  2014-10-07
13356856  zzu6_r3DxBJuXcjnOYVdTw  2012-07-21

[2874514 rows x 2 columns]


In [ ]:
# Elimina las filas duplicadas del DataFrame
checkin_yelp = checkin_yelp.drop_duplicates()

# Muestra el DataFrame sin filas duplicadas
print(checkin_yelp)


                     business_id        date
0         ---kPU91CF4Lq2-WlRu9Lw  2020-03-13
1         ---kPU91CF4Lq2-WlRu9Lw  2020-06-02
2         ---kPU91CF4Lq2-WlRu9Lw  2020-07-24
3         ---kPU91CF4Lq2-WlRu9Lw  2020-10-24
4         ---kPU91CF4Lq2-WlRu9Lw  2020-12-09
...                          ...         ...
13356870  zzu6_r3DxBJuXcjnOYVdTw  2013-12-11
13356871  zzu6_r3DxBJuXcjnOYVdTw  2013-12-13
13356872  zzw66H6hVjXQEt0Js3Mo4A  2016-12-03
13356873  zzw66H6hVjXQEt0Js3Mo4A  2018-12-02
13356874  zzyx5x0Z7xXWWvWnZFuxlQ  2015-01-06

[10482361 rows x 2 columns]


In [ ]:
checkin_yelp.count()

business_id    10482361
date           10482361
dtype: int64

## Carga de nuestro archivo

In [ ]:
# Ruta al archivo CSV
file_path = '/content/drive/MyDrive/Colab-Notebooks/transformaciones/checki-limpio.csv'

# Guardar el DataFrame en un archivo CSV en la carpeta file_path
checkin_yelp.to_csv(file_path, index=False)